# URBANO Y RUSTICO

In [ ]:
import os
import zipfile
import geopandas as gpd
import pandas as pd
import shutil
import subprocess

# 📂 Carpeta base
base_folder = r"C:\a\CARTOGRAFIA_PROVINCIAL_2025"

# 📂 Ruta a 7-Zip
sevenzip_exe = r"C:\Program Files (x86)\7-Zip\7z.exe"

# CRS objetivo
target_crs = "EPSG:25830"

def procesar(tipo, pattern, output_gpkg):
    """
    tipo: 'URBANO' o 'RUSTICO'
    pattern: cadena a buscar ('03_UA' o '03_RA')
    output_gpkg: salida GeoPackage
    """
    print(f"\n=== Procesando {tipo} ===")

    # 1️⃣ Carpeta temporal
    temp_extract = os.path.join(base_folder, f"{tipo.lower()}_temp")
    if os.path.exists(temp_extract):
        shutil.rmtree(temp_extract)
    os.makedirs(temp_extract, exist_ok=True)

    # 2️⃣ Extraer los ZIP externos que contienen el multivolumen real
    for f in os.listdir(base_folder):
        if f.endswith(".zip") and "PETICION_DESCARGA" in f:
            zfile = os.path.join(base_folder, f)
            with zipfile.ZipFile(zfile, "r") as zip_ref:
                zip_ref.extractall(temp_extract)
            print(f"📦 Extraído {zfile} en {temp_extract}")

    # 3️⃣ Localizar multivolumen real
    real_zip = None
    for root, _, files in os.walk(temp_extract):
        for f in files:
            if f.endswith(".zip") and pattern in f:
                real_zip = os.path.join(root, f)
                break
        if real_zip:
            break

    if not real_zip:
        raise RuntimeError(f"❌ No se encontró el multivolumen {pattern}")

    print(f"🔍 Encontrado multivolumen {tipo}: {real_zip}")

    # 4️⃣ Descomprimir multivolumen con 7-Zip
    extract_folder = os.path.join(base_folder, f"{tipo.lower()}_descomprimido")
    if os.path.exists(extract_folder):
        shutil.rmtree(extract_folder)
    os.makedirs(extract_folder, exist_ok=True)

    subprocess.run([sevenzip_exe, "x", "-y", f"-o{extract_folder}", real_zip], check=True)
    print(f"✅ Multivolumen {tipo} descomprimido en: {extract_folder}")

    # 5️⃣ Localizar carpeta con municipios
    municipios_folder = None
    for root, dirs, files in os.walk(extract_folder):
        for d in dirs:
            if d.endswith("_SHP"):
                municipios_folder = os.path.join(root, d)
                break
        if municipios_folder:
            break

    if not municipios_folder:
        raise RuntimeError(f"❌ No se encontró la carpeta de municipios en {tipo}")

    print(f"📂 Carpeta de municipios {tipo} detectada: {municipios_folder}")

    # 6️⃣ Leer shapefiles de cada municipio
    gdfs = []
    for subdir, _, files in os.walk(municipios_folder):
        for file in files:
            if file.endswith("_PARCELA.ZIP"):
                zip_path = os.path.join(subdir, file)
                municipio_name = os.path.basename(subdir)
                extract_path = os.path.join(subdir, "temp_extract")
                os.makedirs(extract_path, exist_ok=True)

                try:
                    with zipfile.ZipFile(zip_path, "r") as zip_ref:
                        zip_ref.extractall(extract_path)

                    shp_path = os.path.join(extract_path, "PARCELA.SHP")
                    if os.path.exists(shp_path):
                        gdf = gpd.read_file(shp_path)
                        if gdf.crs is not None:
                            gdf = gdf.to_crs(target_crs)
                        gdf["MUNICIPIO"] = municipio_name
                        gdfs.append(gdf)
                        print(f"✔ {municipio_name} añadido ({len(gdf)} parcelas)")
                    else:
                        print(f"⚠ No se encontró PARCELA.SHP en {zip_path}")

                except Exception as e:
                    print(f"❌ Error procesando {zip_path}: {e}")

                finally:
                    if os.path.exists(extract_path):
                        shutil.rmtree(extract_path)

    # 7️⃣ Fusionar y guardar en GeoPackage
    if gdfs:
        merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=target_crs)
        merged_gdf.to_file(output_gpkg, driver="GPKG", encoding="utf-8")
        print(f"\n✅ GeoPackage {tipo} creado en: {output_gpkg}")
    else:
        print(f"⚠ No se cargó ningún shapefile en {tipo}.")

    # 8️⃣ Limpiar temporales
    for temp_dir in [temp_extract, extract_folder]:
        if os.path.exists(temp_dir):
            shutil.rmtree(temp_dir)
            print(f"🧹 Carpeta eliminada: {temp_dir}")

# -------------------------------------------------------------------
# 🚀 Ejecutar para URBANO y RÚSTICO
# -------------------------------------------------------------------
procesar("URBANO", "03_UA", os.path.join(base_folder, "URBANO.gpkg"))
procesar("RUSTICO", "03_RA", os.path.join(base_folder, "RUSTICO.gpkg"))



=== Procesando URBANO ===
📦 Extraído C:\a\CARTOGRAFIA_PROVINCIAL_2025\2025092412085607542_PETICION_DESCARGA_SHA.zip en C:\a\CARTOGRAFIA_PROVINCIAL_2025\urbano_temp
📦 Extraído C:\a\CARTOGRAFIA_PROVINCIAL_2025\2025092412230933865_PETICION_DESCARGA_SHA.zip en C:\a\CARTOGRAFIA_PROVINCIAL_2025\urbano_temp
📦 Extraído C:\a\CARTOGRAFIA_PROVINCIAL_2025\2025092412265558173_PETICION_DESCARGA_SHA.zip en C:\a\CARTOGRAFIA_PROVINCIAL_2025\urbano_temp
📦 Extraído C:\a\CARTOGRAFIA_PROVINCIAL_2025\2025092412295903318_PETICION_DESCARGA_SHA.zip en C:\a\CARTOGRAFIA_PROVINCIAL_2025\urbano_temp
🔍 Encontrado multivolumen URBANO: C:\a\CARTOGRAFIA_PROVINCIAL_2025\urbano_temp\03_UA_14082025_SHP.zip
✅ Multivolumen URBANO descomprimido en: C:\a\CARTOGRAFIA_PROVINCIAL_2025\urbano_descomprimido
📂 Carpeta de municipios URBANO detectada: C:\a\CARTOGRAFIA_PROVINCIAL_2025\urbano_descomprimido\03_UA_14082025_SHP
✔ 03001uA 3287 L ATZUBIA añadido (757 parcelas)
✔ 03002uA 3283 AGOST añadido (1599 parcelas)
✔ 03003uA 3209 AG

# FUSION

In [3]:
# Solo elimnando 09000 en rustco

In [4]:
import geopandas as gpd
import pandas as pd
import os

# 📂 Rutas de entrada
urbano_path = r"C:\a\CARTOGRAFIA_PROVINCIAL_2025\URBANO.gpkg"
rustico_path = r"C:\a\CARTOGRAFIA_PROVINCIAL_2025\RUSTICO.gpkg"

# 📂 Ruta de salida
salida_path = r"C:\a\CARTOGRAFIA_PROVINCIAL_2025\PARCELAS_FUSIONADAS.gpkg"

print("📂 Cargando capas...")
urbano = gpd.read_file(urbano_path)
rustico = gpd.read_file(rustico_path)

print(f"🏙️ URBANO: {len(urbano)} parcelas")
print(f"🌾 RÚSTICO (antes de limpiar): {len(rustico)} parcelas")

# ❌ Eliminar de RÚSTICO las parcelas de transición (09000)
rustico_filtrado = rustico[rustico["PARCELA"] != "09000"]

print(f"🌾 RÚSTICO (después de limpiar): {len(rustico_filtrado)} parcelas")

# 🏗 Fusionar URBANO + RÚSTICO filtrado
fusion = gpd.GeoDataFrame(
    pd.concat([urbano, rustico_filtrado], ignore_index=True),
    crs=urbano.crs
)

print(f"✅ Total fusionado: {len(fusion)} parcelas")

# 💾 Guardar en GeoPackage
fusion.to_file(salida_path, driver="GPKG")
print(f"💾 Guardado en {salida_path}")


📂 Cargando capas...
🏙️ URBANO: 405002 parcelas
🌾 RÚSTICO (antes de limpiar): 645290 parcelas
🌾 RÚSTICO (después de limpiar): 540076 parcelas
✅ Total fusionado: 945078 parcelas
💾 Guardado en C:\a\CARTOGRAFIA_PROVINCIAL_2025\PARCELAS_FUSIONADAS.gpkg
